In [3]:
import pandas as pd
import numpy as np

from numpy.linalg import eigh
np.set_printoptions(linewidth=100)
# sample data from 
df = pd.read_csv('/Users/thanakrit.boo/Documents/Local Project/Python/FactorAnalysis/data.csv', index_col=0)

# Data

Five subjects who were trying on ski boots late on a Friday night in January were asked about the importance of each of four variables to their selection of a ski resort. The variables were cost of ski ticket (COST), speed of ski lift (LIFT), depth of snow (DEPTH), and moisture of snow (POWDER). Larger numbers indicate greater importance. The researcher wanted to investigate the pattern of relationships among the variables in an effort to understand better the dimensions underlying choice of ski area.

# Extraction

In [4]:
corr = df.corr()
# Corr from cov, std
# df.cov()/(df.std().to_numpy().reshape(-1,1)@df.std().to_numpy().reshape(1,-1))
corr

,COST,LIFT,DEPTH,POWDER
COST,1.000000,-0.952990,-0.055276,-0.129999
LIFT,-0.952990,1.000000,-0.091107,-0.036248
DEPTH,-0.055276,-0.091107,1.000000,0.990174
POWDER,-0.129999,-0.036248,0.990174,1.000000


In [5]:
eigval, eigvec = eigh(corr)
display(eigval)
display(eigvec)

array([0.00436878, 0.03781231, 1.94151381, 2.0163051 ])

array([[ 0.24394508,  0.6624913 , -0.61432982,  0.35241302],
       [ 0.1988    ,  0.67589344,  0.66376422, -0.25112482],
       [-0.65319195,  0.2754625 , -0.32222906, -0.6273987 ],
       [ 0.68870141, -0.16850441, -0.27961467, -0.6473888 ]])

In [6]:
# Re-arrage of correlation matrix from eigen value, eigen vector
re_composed_corr = eigvec@np.diag(eigval)@eigvec.T
display(re_composed_corr)

array([[ 1.        , -0.95299048, -0.05527555, -0.12999882],
       [-0.95299048,  1.        , -0.09110654, -0.03624823],
       [-0.05527555, -0.09110654,  1.        ,  0.99017435],
       [-0.12999882, -0.03624823,  0.99017435,  1.        ]])

In [7]:
# Split eigval into sqrt(np.diag(eigval))*sqrt(np.diag(eigval)), call
# eigvec@(np.diag(eigval)) = (unrotated) factor loading matrix
factor_loading = eigvec@np.diag(np.sqrt(eigval))
display(factor_loading)

array([[ 0.01612397,  0.128824  , -0.85599621,  0.50041471],
       [ 0.01314003,  0.13143009,  0.92487722, -0.35658885],
       [-0.04317384,  0.05356475, -0.44898822, -0.89088519],
       [ 0.0455209 , -0.03276633, -0.38961008, -0.91927047]])

To represent the correlation matrix, with less number of factors.  
Use high value eigen value (and corresponding eigen vector), since higher eigen value, shown relation between factor and original correlation matrix.  
If choose only top 2 factor

In [8]:
top_eigval = eigval[:-3:-1] # re-order by top value first
top_eigval

array([2.0163051 , 1.94151381])

In [9]:
top_eigvec = eigvec[:,[3,2]] # re-order according to corresponding eigen val
top_eigvec

array([[ 0.35241302, -0.61432982],
       [-0.25112482,  0.66376422],
       [-0.6273987 , -0.32222906],
       [-0.6473888 , -0.27961467]])

Reproduce correlation matrix from first 2 factos

In [10]:
top_corr = top_eigvec@np.diag(top_eigval)@top_eigvec.T
top_corr

array([[ 0.9831444 , -0.9701337 , -0.06147984, -0.12651171],
       [-0.9701337 ,  0.98255347, -0.09757925, -0.03253989],
       [-0.06147984, -0.09757925,  0.99526684,  0.99389478],
       [-0.12651171, -0.03253989,  0.99389478,  0.99685421]])

In [11]:
corr

,COST,LIFT,DEPTH,POWDER
COST,1.000000,-0.952990,-0.055276,-0.129999
LIFT,-0.952990,1.000000,-0.091107,-0.036248
DEPTH,-0.055276,-0.091107,1.000000,0.990174
POWDER,-0.129999,-0.036248,0.990174,1.000000


Top Factor loading = top_eigvec @ sqrt(eigval)

In [12]:
top_factor_loading = top_eigvec@np.diag(np.sqrt(top_eigval))
top_factor_loading

array([[ 0.50041471, -0.85599621],
       [-0.35658885,  0.92487722],
       [-0.89088519, -0.44898822],
       [-0.91927047, -0.38961008]])

__Interpretation__  
First factor high relation with `DEPTH` and `POWDER`, indicate the factor based on environment  
Second factor high relation with `COST` and `LIFT` , indicate the factor based on infrastructure  

# Orthogonal Rotation  
Rotation to maximized high correlation factor <-> variable  
_varimax_ high factor loading -> higher / low factor loading -> lower

`varimax`  
orthogonal rotate with matrix
| cos x | - sin x |
| --- | --- |
| sin x | cos x |

In [13]:
rotate_radian = 19/180*np.pi
rotate_mat = np.array([[np.cos(rotate_radian), -np.sin(rotate_radian)],
                       [np.sin(rotate_radian), np.cos(rotate_radian)]])
top_factor_loading_rotated = top_factor_loading@rotate_mat
top_factor_loading_rotated

array([[ 0.1944663 , -0.97227941],
       [-0.03605081,  0.99058256],
       [-0.98852476, -0.13448285],
       [-0.99603194, -0.06909838]])

## Varimax rotation
1) Normalized each row with each row sum-square

In [42]:
# sample Factor loading sample for calculation reference : https://www.real-statistics.com/linear-algebra-matrix-topics/varimax/
df = pd.read_excel('/Users/thanakrit.boo/Documents/Local Project/Python/FactorAnalysis/sample_cal_varimax.xlsx',header=None)
top_factor_loading = df.to_numpy()

In [37]:
def varimax(X):
    """X is k by 2 matrix
    """
    u = X[:,0]**2 - X[:,1]**2
    # print(u)
    v = 2*(X[:,0]*X[:,1])
    # print(v)
    u_sqrt_net_v_sqrt = u**2 - v**2
    # print(u_sqrt_net_v_sqrt)
    uv = u*v
    # print(uv)

    A = u.sum()
    B = v.sum()
    C = u_sqrt_net_v_sqrt.sum()
    D = uv.sum()
    k = X.shape[0]

    X = 2*D*k - 2*A*B 
    Y = C*k - (A**2 - B**2)
    # print(X)
    # print(Y)

    rotate_radian = np.arctan(X/Y)*0.25
    rotate_mat = np.array([[np.cos(rotate_radian), - np.sin(rotate_radian)],
                       [np.sin(rotate_radian), np.cos(rotate_radian)]])
    return rotate_radian, rotate_mat
    

In [58]:
# top_factor_loading = top_eigvec@np.diag(np.sqrt(top_eigval))
print(top_factor_loading)
communality = (top_factor_loading**2).sum(axis=1).reshape(-1,1)
normalized = top_factor_loading/np.sqrt(communality) #.reshape(-1,1))
print(normalized)

MAX_ROUND = 5
N_COL = top_factor_loading.shape[1]
for r in range(0, MAX_ROUND):
    for i in range(0, N_COL):
        for j in range(i+1, N_COL):
            by2 = normalized[:, [i, j]]
            rot_radian, rot_mat = varimax(by2)
            print(f'round {i, j} : {rot_radian}')
            by2_rot = by2@rot_mat
            normalized[:, [i,j]] = by2_rot
            # print(normalized)
unnormalized = normalized*np.sqrt(communality)
print(unnormalized)

[[-0.18444    0.76673   -0.27758    0.115158 ]
 [ 0.698485  -0.30362   -0.19539   -0.2648   ]
 [ 0.754094   0.348652  -0.20205   -0.30588  ]
 [ 0.365978   0.162525  -0.19731    0.849845 ]
 [ 0.578163   0.04413    0.452722   0.157895 ]
 [ 0.299667   0.0303881  0.770856   0.063739 ]
 [ 0.817187   0.239353  -0.19451   -0.16643  ]
 [ 0.687893   0.035304   0.116032   0.117627 ]
 [ 0.301458  -0.74108   -0.3442     0.233979 ]]
[[-0.2185509   0.90853139 -0.3289165   0.13645567]
 [ 0.84187657 -0.36594997 -0.2355015  -0.31916063]
 [ 0.83043012  0.38394567 -0.2225033  -0.3368439 ]
 [ 0.38124321  0.16930404 -0.20553994  0.88529266]
 [ 0.76842355  0.0586522   0.60170272  0.20985472]
 [ 0.36101697  0.03660937  0.92867114  0.0767881 ]
 [ 0.91904807  0.26918797 -0.21875536 -0.18717524]
 [ 0.97113669  0.04984061  0.16380881  0.16606056]
 [ 0.33427386 -0.82175186 -0.38166863  0.25944929]]
round (0, 1) : 0.268260641029102
round (0, 2) : -0.3285740477653107
round (0, 3) : -0.3670188335558344
round (1, 2) 